In [1]:
import pandas as pd
from ptrail.core.TrajectoryDF import PTRAILDataFrame
from ptrail.preprocessing.filters import Filters as filters
from ptrail.features.kinematic_features import KinematicFeatures
from ptrail.features.temporal_features import TemporalFeatures
from ptrail.preprocessing.interpolation import Interpolation
from ptrail.utilities.conversions import Conversions as con
import ptrail.utilities.constants as const
import datetime as dt

# Reading the gulls dataset and converting to PTRAILDataFrame.
# Also, lets, print the first 5 points of the dataset to
# see how the dataframe looks.
gulls = pd.read_csv('https://raw.githubusercontent.com/YakshHaranwala/PTRAIL/main/examples/data/gulls.csv')
gulls = PTRAILDataFrame(gulls,
                        latitude='location-lat',
                        longitude='location-long',
                        datetime='timestamp',
                        traj_id='tag-local-identifier',
                        rest_of_columns=[])

In [2]:
pdf = pd.read_csv('https://raw.githubusercontent.com/YakshHaranwala/PTRAIL/main/examples/data/geolife_sample.csv')
geolife = PTRAILDataFrame(data_set=pdf,
                             latitude='lat',
                             longitude='lon',
                             datetime='datetime',
                             traj_id='id')


In [3]:
atlantic = pd.read_csv('https://raw.githubusercontent.com/YakshHaranwala/PTRAIL/main/examples/data/atlantic.csv')
atlantic = con.convert_directions_to_degree_lat_lon(atlantic, 'Latitude',"Longitude")
def convert_to_datetime(row):
        this_date = '{}-{}-{}'.format(str(row['Date'])[0:4], str(row['Date'])[4:6], str(row['Date'])[6:])
        this_time = '{:02d}:{:02d}:00'.format(int(row['Time']/100), int(str(row['Time'])[-2:]))
        return '{} {}'.format(this_date, this_time)
atlantic['DateTime'] = atlantic.apply(convert_to_datetime, axis=1)
atlantic = PTRAILDataFrame(atlantic,
                          latitude='Latitude',
                          longitude='Longitude',
                          datetime='DateTime',
                          traj_id='ID',
                          rest_of_columns=[])
atlantic = TemporalFeatures.create_date_column(atlantic)

In [4]:
pdf = pd.read_csv('https://raw.githubusercontent.com/YakshHaranwala/PTRAIL/main/examples/data/starkey.csv')
starkey = PTRAILDataFrame(data_set=pdf,
                          latitude='lat',
                          longitude='lon',
                          datetime='DateTime',
                          traj_id='Id')

In [5]:
t1 = gulls.reset_index().loc[gulls.reset_index()[const.TRAJECTORY_ID] == '91732'][[const.TRAJECTORY_ID, const.DateTime, const.LAT, const.LONG]]
t1 = PTRAILDataFrame(data_set=t1,
                     datetime='DateTime',
                     traj_id='traj_id',
                     latitude='lat',
                     longitude='lon')

In [6]:
def segment_traj_by_week(df):
    """
        Given a dataframe containing trajectory data, segment all
        the trajectories by each week.

        Parameters
        ----------
            df: PTRAILDataFrame
                The dataframe containing trajectory data.

        Returns
        -------
            pandas.core.dataframe.DataFrame:
                The dataframe containing segmented trajectories
                with a new column added called segment_id
    """
    # First, create the date column and get all the unique traj_ids
    # in the dataframe..
    df = TemporalFeatures.create_date_column(df)
    ids_ = list(df.traj_id.value_counts().keys())

    # Get the ideal number of IDs by which the dataframe is to be split.
    df_chunks = []
    for i in range(len(ids_)):
        small_df = df.reset_index().loc[df.reset_index()[const.TRAJECTORY_ID] == ids_[i]]
        df_chunks.append(small_df)

    # Now, iterate over the entire dataframe and then segment
    # the trajectories by 1 week each.
    results = []
    for i in range(len(ids_)):
        # Take the traj_df of a single Trajectory out from the
        # list of chunks and find their max and min timestamps.
        traj = df_chunks[i]
        t_max = traj.reset_index()[const.DateTime].max()
        t_min = traj.reset_index()[const.DateTime].min()

        # For iteration purposes, set t_1 to min and t_2 to
        # t_1 + 7 days.
        t_1 = t_min
        t_2 = t_1 + dt.timedelta(days=7)
        seg_id = 1

        # Now, segment the trajectories into smaller segments
        # wherein each segment contains the points of a span
        # of 7 days only.
        while t_2 < t_max:
            if t_2 < t_max:
                seg = filters.filter_by_date(traj,
                                             start_date=t_1.strftime('%Y-%m-%d'),
                                             end_date=t_2.strftime('%Y-%m-%d'))
                # Once filtered, assign the segment with a segment ID.
                seg['seg_id'] = seg_id

                # Increment the segment id, t_1 and t_2 values by
                # 1, 7 days and 7 days respectively to continue
                # the iteration.
                t_1 += dt.timedelta(days=7)
                t_2 += dt.timedelta(days=7)
                results.append(seg.drop(columns=['index', 'level_0']))

            # If, t_2 is greater than the max time present in the
            # trajectory, then assign t_2 = max and proceed
            # further with segmentation.
            elif t_2 >= t_max:
                seg = filters.filter_by_date(traj,
                                             start_date=t_1.strftime('%Y-%m-%d'),
                                             end_date=t_max.strftime('%Y-%m-%d'))
                # Once filtered, assign the segment with a segment ID.
                seg['seg_id'] = seg_id

                # Increment the segment id, t_1 and t_2 values by
                # 1, 7 days and 7 days respectively to continue
                # the iteration.
                t_1 += dt.timedelta(days=7)
                t_2 += dt.timedelta(days=7)
                results.append(seg.drop(columns=['index', 'level_0']))
            seg_id += 1

    # Finally, concat the dataframes, set the index as
    # [traj_id, seg_id, DateTime].
    return pd.concat(results).reset_index().set_index(['traj_id', 'seg_id', 'DateTime'])

In [7]:
%%time

seg = segment_traj_by_week(starkey)

CPU times: user 3min 44s, sys: 416 ms, total: 3min 44s
Wall time: 3min 44s


In [8]:
print(len(starkey))

287136


In [9]:
%%time

# Now, generating the stats of kinematic features for
# all the segments of the trajectories.

# First, splitting the dataframe into several parts
# wherein each part has a segment of the trajectory only.
ids_ = list(seg.reset_index()['traj_id'].value_counts().keys())

df_chunks = []
for i in range(len(ids_)):
    small_df = seg.reset_index().loc[seg.reset_index()[const.TRAJECTORY_ID] == ids_[i]]
    df_chunks.append(small_df)

final_chunks = []
for i in range(len(ids_)):
    seg_ids = list(df_chunks[i]['seg_id'].value_counts().keys())
    for j in range(len(seg_ids)):
        mini_df = df_chunks[i].loc[df_chunks[i]['seg_id'] == seg_ids[j]]
        final_chunks.append(mini_df)

print(len(final_chunks))

4976
CPU times: user 53.4 s, sys: 7.59 ms, total: 53.4 s
Wall time: 53.5 s


In [10]:
def segment_features(df):
    """
        Generate all the kinematic features on the smaller segments.

        Parameters
        ----------
            df: PTRAILDataFrame/pd.DataFrame
                The dataframe containing the trajectory segment.

        Returns
        -------
            pd.core.dataframe.DataFrame:
                The dataframe containing the segment kinematic
                feature stats.
    """
    seg_id = final_chunks[i]['seg_id'].iloc[0]
    stat_df = KinematicFeatures.generate_kinematic_stats(df)
    stat_df['seg_id'] = seg_id
    stat_df = stat_df.reset_index().set_index(['traj_id', 'seg_id', 'Columns'])

    return stat_df

In [11]:
%%time
# Now, generating kinematic features and their stats on each of
# the dataframe here.
import progressbar

final_results = []
bar = progressbar.ProgressBar(max_value=progressbar.UnknownLength)
for i in range(len(final_chunks)):
    final_results.append(segment_features(final_chunks[i]))
    bar.update(i)

final_answer = pd.concat(final_results)

| |                                  #             | 4975 Elapsed Time: 0:23:08

CPU times: user 11min 48s, sys: 7min 19s, total: 19min 7s
Wall time: 23min 10s


In [12]:
final_answer.head(40)

mean           std         min  \
traj_id   seg_id Columns                                                        
910313E37 1      Distance              1.709843e+02  2.741726e+02    0.000000   
                 Distance_from_start   3.145115e+03  1.134059e+03   67.090679   
                 Speed                 1.282633e-01  3.343164e-01    0.000000   
                 Acceleration          5.894095e-05  5.653138e-04   -0.001597   
                 Jerk                  1.785475e-07  1.896180e-06   -0.000006   
                 Bearing               1.704456e+02  1.070074e+02    0.000000   
                 Bearing_Rate         -3.246468e-03  1.582499e-01   -0.845313   
                 Rate_of_bearing_rate -3.246468e-03  1.582499e-01   -0.845313   
          3      Distance              2.133147e+02  3.626186e+02    0.000000   
                 Distance_from_start   2.246943e+03  1.040147e+03  108.142513   
                 Speed                 1.442296e-01  4.927638e-01    0.000000   
                 Acceleration          6.533882e-04  7.686860e-03   -0.002575   
                 Jerk                  1.582286e-05  2.216047e-04   -0.000060   
                 Bearing               1.824146e+02  1.059418e+02    0.000000   
                 Bearing_Rate         -2.681183e-02  4.021768e-01   -6.036678   
                 Rate_of_bearing_rate -2.681183e-02  4.021768e-01   -6.036678   
          6      Distance              2.182084e+02  3.216624e+02    0.000000   
                 Distance_from_start   1.537511e+03  8.374627e+02   89.766532   
                 Speed                 9.241706e-02  2.667963e-01    0.000000   
                 Acceleration          4.577331e-05  5.668969e-04   -0.000703   
                 Jerk                  1.178590e-07  1.371983e-06   -0.000002   
                 Bearing               1.697428e+02  1.020041e+02    0.000000   
                 Bearing_Rate         -9.533754e-04  8.757836e-02   -0.586953   
                 Rate_of_bearing_rate -9.533754e-04  8.757836e-02   -0.586953   
          4      Distance              1.045948e+02  7.927141e+01    0.000000   
                 Distance_from_start   1.957693e+02  1.596711e+02    0.000000   
                 Speed                 5.233723e-02  5.479076e-02    0.000000   
                 Acceleration          3.169289e-05  4.241867e-04   -0.000153   
                 Jerk                  3.620824e-07  5.228997e-06   -0.000002   
                 Bearing               1.701980e+02  1.056060e+02    0.000000   
                 Bearing_Rate          1.149797e-02  1.903723e-01   -0.188564   
                 Rate_of_bearing_rate  1.149797e-02  1.903723e-01   -0.188564   
          2      Distance              2.724507e+02  4.480825e+02    0.000000   
                 Distance_from_start   1.557462e+03  1.139460e+03   42.384583   
                 Speed                 1.789963e-01  4.595826e-01    0.000000   
                 Acceleration          4.468955e-05  4.477531e-04   -0.001025   
                 Jerk                  3.670608e-08  7.275207e-07   -0.000004   
                 Bearing               1.672029e+02  1.102381e+02    0.000000   
                 Bearing_Rate         -2.143388e-03  8.503347e-02   -0.292818   
                 Rate_of_bearing_rate -2.143388e-03  8.503347e-02   -0.292818   

                                                10%           25%  \
traj_id   seg_id Columns                                            
910313E37 1      Distance              2.992242e+01  4.239008e+01   
                 Distance_from_start   1.306805e+03  2.670747e+03   
                 Speed                 1.679873e-02  2.441513e-02   
                 Acceleration         -6.218602e-05 -1.880435e-05   
                 Jerk                 -5.969154e-08 -1.442411e-08   
                 Bearing               3.250344e+01  7.754472e+01   
                 Bearing_Rate         -1.282694e-01 -6.152395e-02   
                 Rate_o

In [ ]:

#